# Numerical Data - Cleanup

<br>
**Author** Mary Letey <br>
**Date** 3/20/18 <br>
**File** numclean.ipynb <br>
**Purpose** Formating numerical (Bloomberg) data

<br><br>

In [5]:
# Imports
import numpy as np
import pandas as pd
import datetime
import matplotlib.pylab as plt
%matplotlib inline 

In [50]:
def getDates(rowlist, dates):
    datestring = rowlist[1]
    datelist = datestring.split('/')
    year = int(datelist[2])
    if (int(year) > 20):
        year += 1900
    else:
        year += 2000
    datelist[2] = str(year)
    old_datelist = datelist
    datelist = ''
    for j in range(0,len(old_datelist)):
        datelist += old_datelist[j]
        if j < len(old_datelist)-1:
            datelist += '/'
    dateobject = datetime.datetime.strptime(datelist,'%m/%d/%Y')
    dates.append(dateobject)    
    return dates

def getVols(rowlist, vols):
    
    volstring = rowlist[3]
    
    vol = 0
    
    if ('M' in volstring) or ('m' in volstring):
        if 'M' in volstring:
            vollist = volstring.split('M')
        else:
            vollist = volstring.split('m')
        vol = float(vollist[0])
        vol *= 1000000
        vols.append(vol)
        return vols
    elif ('K' in volstring) or ('k' in volstring):
        if 'K' in volstring:
            vollist = volstring.split('K')
        else:
            vollist = volstring.split('k')
        vol = float(vollist[0])
        vol *= 1000
        vols.append(vol)
        return vols
    elif list(volstring) == []:
        vol = 0
        vols.append(vol)
        return vols
    else:
        vol = int(volstring)
        vols.append(vol)
        return vols
    
    print("oopsies", volstring)
    return vols

def cleanData(files):
    dates = []
    spots = []
    vols = []

    for i in range(1,len(files)):
        row = files[i]
        rowlist = row.split(',')
        if rowlist != ['', '', '', '', ''] and rowlist != [' ', ' ', ' ', ' ', ' ']:
            dates = getDates(rowlist, dates)
            print("spot of",rowlist[2],"with type",type(rowlist[2]))
            spots.append(float(rowlist[2]))
            vols = getVols(rowlist,vols)
    
    return dates, spots, vols

In [ ]:
#hp
hp_path = "../datafiles/numerical-initial/hp-data.csv"
hp_files = []
for hp_file in open(hp_path,"r"):
    hp_files.append(hp_file.split('\n')[0])
hp_dates, hp_spots, hp_vols = cleanData(hp_files)  

#ibm
ibm_path = "../datafiles/numerical-initial/ibm-data.csv"
ibm_files = []
for ibm_file in open(ibm_path,"r"):
    ibm_files.append(ibm_file.split('\n')[0])
ibm_dates, ibm_spots, ibm_vols = cleanData(ibm_files)

#dell
dell_path = "../datafiles/numerical-initial/dell-data.csv"
dell_files = []
for dell_file in open(dell_path,"r"):
    dell_files.append(dell_file.split('\n')[0])
dell_dates, dell_spots, dell_vols = cleanData(dell_files)

#seagate
seagate_path = "../datafiles/numerical-initial/seagate-data.csv"
seagate_files = []
for seagate_file in open(seagate_path,"r"):
    seagate_files.append(seagate_file.split('\n')[0])
seagate_dates, seagate_spots, seagate_vols = cleanData(seagate_files)

#westdig
westdig_path = "../datafiles/numerical-initial/westdig-data.csv"
westdig_files = []
for westdig_file in open(westdig_path,"r"):
    westdig_files.append(westdig_file.split('\n')[0])
westdig_dates, westdig_spots, westdig_vols = cleanData(westdig_files)

#westdig
dow_path = "../datafiles/numerical-initial/dow-data.csv"
dow_files = []
for dow_file in open(dow_path,"r"):
    dow_files.append(dow_file.split('\n')[0])
print(dow_files)
dow_dates, dow_spots, dow_vols = cleanData(dow_files)

ibm_df = pd.DataFrame({'dates': ibm_dates, 'IBM spots': ibm_spots, 'IBM vols': ibm_vols})
ibm_df.head()

dell_df = pd.DataFrame({'dates': dell_dates, 'DELL spots': dell_spots, 'DELL vols': dell_vols})
dell_df.head()

hp_df = pd.DataFrame({'dates': hp_dates, 'HP spots': hp_spots, 'HP vols': hp_vols})
hp_df.head()

westdig_df = pd.DataFrame({'dates': westdig_dates, 'WESTDIG spots': westdig_spots, 'WESTDIG vols': westdig_vols})
westdig_df.head()

seagate_df = pd.DataFrame({'dates': seagate_dates, 'SEAGATE spots': seagate_spots, 'SEAGATE vols': seagate_vols})
seagate_df.head()

dow_df = pd.DataFrame({'dates': dow_dates, 'DOW spots': dow_spots, 'DOW vols': dow_vols})
dow_df.head()

total_df = pd.merge(ibm_df, dell_df, on="dates")
total_df = pd.merge(total_df, hp_df, on="dates")
total_df = pd.merge(total_df, westdig_df, on="dates")
total_df = pd.merge(total_df, seagate_df, on="dates")
total_df = pd.merge(total_df, dow_df, on="dates")

pd.DataFrame.to_csv(total_df, "./cleanedNumData.csv")

spot of 23.2 with type <class 'str'>
spot of 23.51 with type <class 'str'>
spot of 23.48 with type <class 'str'>
spot of 23.48 with type <class 'str'>
spot of 23.62 with type <class 'str'>
spot of 23.83 with type <class 'str'>
spot of 24.65 with type <class 'str'>
spot of 23.95 with type <class 'str'>
spot of 24.04 with type <class 'str'>
spot of 23.95 with type <class 'str'>
spot of 23.69 with type <class 'str'>
spot of 23.57 with type <class 'str'>
spot of 23.35 with type <class 'str'>
spot of 23.39 with type <class 'str'>
spot of 23.52 with type <class 'str'>
spot of 23.46 with type <class 'str'>
spot of 22.13 with type <class 'str'>
spot of 21.39 with type <class 'str'>
spot of 21.17 with type <class 'str'>
spot of 21.39 with type <class 'str'>
spot of 21.4 with type <class 'str'>
spot of 21.45 with type <class 'str'>
spot of 21.29 with type <class 'str'>
spot of 20.58 with type <class 'str'>
spot of 20.41 with type <class 'str'>
spot of 20.11 with type <class 'str'>
spot of 19.92 

spot of 11.55 with type <class 'str'>
spot of 12.26 with type <class 'str'>
spot of 12.95 with type <class 'str'>
spot of 12.61 with type <class 'str'>
spot of 13.33 with type <class 'str'>
spot of 13.07 with type <class 'str'>
spot of 13.12 with type <class 'str'>
spot of 12.98 with type <class 'str'>
spot of 13 with type <class 'str'>
spot of 13.05 with type <class 'str'>
spot of 13.03 with type <class 'str'>
spot of 13.5 with type <class 'str'>
spot of 13.84 with type <class 'str'>
spot of 13.87 with type <class 'str'>
spot of 13.69 with type <class 'str'>
spot of 13.67 with type <class 'str'>
spot of 13.44 with type <class 'str'>
spot of 13.54 with type <class 'str'>
spot of 13.44 with type <class 'str'>
spot of 13.38 with type <class 'str'>
spot of 13.17 with type <class 'str'>
spot of 13.04 with type <class 'str'>
spot of 12.2 with type <class 'str'>
spot of 11.91 with type <class 'str'>
spot of 11.67 with type <class 'str'>
spot of 11.66 with type <class 'str'>
spot of 11.44 wit

spot of 13.6247 with type <class 'str'>
spot of 13.6066 with type <class 'str'>
spot of 13.8608 with type <class 'str'>
spot of 14.1695 with type <class 'str'>
spot of 14.3193 with type <class 'str'>
spot of 14.3602 with type <class 'str'>
spot of 14.392 with type <class 'str'>
spot of 14.4374 with type <class 'str'>
spot of 14.5645 with type <class 'str'>
spot of 14.6235 with type <class 'str'>
spot of 14.3693 with type <class 'str'>
spot of 14.3965 with type <class 'str'>
spot of 14.7143 with type <class 'str'>
spot of 14.7643 with type <class 'str'>
spot of 14.9413 with type <class 'str'>
spot of 14.7915 with type <class 'str'>
spot of 14.8415 with type <class 'str'>
spot of 14.9459 with type <class 'str'>
spot of 15.1184 with type <class 'str'>
spot of 15.418 with type <class 'str'>
spot of 15.3817 with type <class 'str'>
spot of 15.3272 with type <class 'str'>
spot of 15.1638 with type <class 'str'>
spot of 15.2773 with type <class 'str'>
spot of 15.3363 with type <class 'str'>
sp

spot of 15.3817 with type <class 'str'>
spot of 15.5179 with type <class 'str'>
spot of 15.2909 with type <class 'str'>
spot of 15.3953 with type <class 'str'>
spot of 15.3953 with type <class 'str'>
spot of 15.2501 with type <class 'str'>
spot of 15.4135 with type <class 'str'>
spot of 15.6723 with type <class 'str'>
spot of 15.5361 with type <class 'str'>
spot of 15.6541 with type <class 'str'>
spot of 15.7585 with type <class 'str'>
spot of 15.8493 with type <class 'str'>
spot of 15.8448 with type <class 'str'>
spot of 15.9628 with type <class 'str'>
spot of 15.1593 with type <class 'str'>
spot of 15.0957 with type <class 'str'>
spot of 15.2591 with type <class 'str'>
spot of 15.3182 with type <class 'str'>
spot of 15.3636 with type <class 'str'>
spot of 15.3772 with type <class 'str'>
spot of 15.3272 with type <class 'str'>
spot of 15.2955 with type <class 'str'>
spot of 15.1774 with type <class 'str'>
spot of 15.2092 with type <class 'str'>
spot of 15.2728 with type <class 'str'>


spot of 11.6135 with type <class 'str'>
spot of 11.4319 with type <class 'str'>
spot of 11.3592 with type <class 'str'>
spot of 11.3184 with type <class 'str'>
spot of 11.2593 with type <class 'str'>
spot of 11.2457 with type <class 'str'>
spot of 10.9007 with type <class 'str'>
spot of 10.828 with type <class 'str'>
spot of 10.6374 with type <class 'str'>
spot of 10.9642 with type <class 'str'>
spot of 11.223 with type <class 'str'>
spot of 11.5454 with type <class 'str'>
spot of 11.5499 with type <class 'str'>
spot of 11.4228 with type <class 'str'>
spot of 11.2321 with type <class 'str'>
spot of 11.3184 with type <class 'str'>
spot of 11.3093 with type <class 'str'>
spot of 11.0051 with type <class 'str'>
spot of 11.1186 with type <class 'str'>
spot of 11.2639 with type <class 'str'>
spot of 11.0096 with type <class 'str'>
spot of 10.9824 with type <class 'str'>
spot of 11.1776 with type <class 'str'>
spot of 11.3047 with type <class 'str'>
spot of 11.0868 with type <class 'str'>
sp

...

spot of 11.0596 with type <class 'str'>
spot of 11.0323 with type <class 'str'>
spot of 10.9779 with type <class 'str'>
spot of 11.0278 with type <class 'str'>
spot of 11.2276 with type <class 'str'>
spot of 11.1322 with type <class 'str'>
spot of 11.3138 with type <class 'str'>
spot of 11.3229 with type <class 'str'>
spot of 11.1776 with type <class 'str'>
spot of 11.0868 with type <class 'str'>
spot of 10.9461 with type <class 'str'>
spot of 10.9552 with type <class 'str'>
spot of 11.0233 with type <class 'str'>
spot of 10.9642 with type <class 'str'>
spot of 10.878 with type <class 'str'>
spot of 10.7645 with type <class 'str'>
spot of 10.6601 with type <class 'str'>
spot of 10.6737 with type <class 'str'>
spot of 10.9279 with type <class 'str'>
spot of 10.7418 with type <class 'str'>
spot of 10.6737 with type <class 'str'>
spot of 10.8053 with type <class 'str'>
spot of 11.0732 with type <class 'str'>
spot of 11.1231 with type <class 'str'>
spot of 11.1731 with type <class 'str'>
s

spot of 9.1755 with type <class 'str'>
spot of 9.0483 with type <class 'str'>
spot of 9.0801 with type <class 'str'>
spot of 8.9575 with type <class 'str'>
spot of 8.8894 with type <class 'str'>
spot of 9.062 with type <class 'str'>
spot of 8.9394 with type <class 'str'>
spot of 9.0801 with type <class 'str'>
spot of 8.9666 with type <class 'str'>
spot of 8.9167 with type <class 'str'>
spot of 8.9212 with type <class 'str'>
spot of 9.0347 with type <class 'str'>
spot of 9.1936 with type <class 'str'>
spot of 9.1301 with type <class 'str'>
spot of 9.1391 with type <class 'str'>
spot of 9.0756 with type <class 'str'>
spot of 9.1982 with type <class 'str'>
spot of 9.0983 with type <class 'str'>
spot of 9.2663 with type <class 'str'>
spot of 9.3434 with type <class 'str'>
spot of 9.5795 with type <class 'str'>
spot of 9.5795 with type <class 'str'>
spot of 9.525 with type <class 'str'>
spot of 9.4479 with type <class 'str'>
spot of 9.5432 with type <class 'str'>
spot of 9.5931 with type <c

spot of 9.6113 with type <class 'str'>
spot of 9.5432 with type <class 'str'>
spot of 9.6567 with type <class 'str'>
spot of 9.7066 with type <class 'str'>
spot of 9.7157 with type <class 'str'>
spot of 9.5795 with type <class 'str'>
spot of 10.0472 with type <class 'str'>
spot of 9.9745 with type <class 'str'>
spot of 9.9109 with type <class 'str'>
spot of 10.2378 with type <class 'str'>
spot of 9.9019 with type <class 'str'>
spot of 10.3059 with type <class 'str'>
spot of 10.3967 with type <class 'str'>
spot of 10.6782 with type <class 'str'>
spot of 10.3786 with type <class 'str'>
spot of 10.0108 with type <class 'str'>
spot of 10.4104 with type <class 'str'>
spot of 10.0381 with type <class 'str'>
spot of 10.0108 with type <class 'str'>
spot of 9.6703 with type <class 'str'>
spot of 9.7884 with type <class 'str'>
spot of 9.6158 with type <class 'str'>
spot of 9.6703 with type <class 'str'>
spot of 9.4842 with type <class 'str'>
spot of 9.6703 with type <class 'str'>
spot of 9.4433 

spot of 6.4741 with type <class 'str'>
spot of 6.8101 with type <class 'str'>
spot of 6.9327 with type <class 'str'>
spot of 6.9145 with type <class 'str'>
spot of 6.9236 with type <class 'str'>
spot of 7.1824 with type <class 'str'>
spot of 7.2732 with type <class 'str'>
spot of 7.4003 with type <class 'str'>
spot of 6.91 with type <class 'str'>
spot of 6.6376 with type <class 'str'>
spot of 6.8555 with type <class 'str'>
spot of 6.9372 with type <class 'str'>
spot of 6.8918 with type <class 'str'>
spot of 6.7284 with type <class 'str'>
spot of 6.8055 with type <class 'str'>
spot of 7.2187 with type <class 'str'>
spot of 7.4729 with type <class 'str'>
spot of 7.5592 with type <class 'str'>
spot of 7.6954 with type <class 'str'>
spot of 8.1131 with type <class 'str'>
spot of 8.0541 with type <class 'str'>
spot of 7.877 with type <class 'str'>
spot of 7.9451 with type <class 'str'>
spot of 8.1539 with type <class 'str'>
spot of 8.331 with type <class 'str'>
spot of 8.3038 with type <cla

spot of 11.5817 with type <class 'str'>
spot of 11.1958 with type <class 'str'>
spot of 10.9915 with type <class 'str'>
spot of 11.0596 with type <class 'str'>
spot of 10.8961 with type <class 'str'>
spot of 11.6589 with type <class 'str'>
spot of 11.6907 with type <class 'str'>
spot of 11.895 with type <class 'str'>
spot of 11.9948 with type <class 'str'>
spot of 12.0357 with type <class 'str'>
spot of 11.9131 with type <class 'str'>
spot of 11.9858 with type <class 'str'>
spot of 11.9858 with type <class 'str'>
spot of 12.7031 with type <class 'str'>
spot of 12.2672 with type <class 'str'>
spot of 11.7088 with type <class 'str'>
spot of 11.5545 with type <class 'str'>
spot of 12.122 with type <class 'str'>
spot of 11.9994 with type <class 'str'>
spot of 12.5442 with type <class 'str'>
spot of 12.8938 with type <class 'str'>
spot of 13.03 with type <class 'str'>
spot of 12.9846 with type <class 'str'>
spot of 12.3717 with type <class 'str'>
spot of 12.0085 with type <class 'str'>
spot

In [45]:
print(rowlist)

NameError: name 'rowlist' is not defined